# 2022-09-29 • Use parametric STA model for connection testing

## Imports

In [3]:
#

In [4]:
using MyToolbox

In [6]:
using VoltoMapSim

[ Info: Precompiling VoltoMapSim [f713100b-c48c-421a-b480-5fcb4c589a9e]


Note that we consolidated code from the previous notebook in the codebase (see github, `pkg/VoltoMapSim/src/`)

## Params

In [7]:
p = get_params(
    duration = 10minutes,
    p_conn = 0.04,
    g_EE = 1,
    g_EI = 1,
    g_IE = 4,
    g_II = 4,
    ext_current = Normal(-0.5 * pA/√seconds, 5 * pA/√seconds),
    E_inh = -80 * mV,
    record_v = [1:40; 801:810],
);

## Load STA's

(They're precalculated).

In [51]:
out = cached_STAs(p);

Loading cached output from `C:\Users\tfiers\.phdcache\calc_all_STAs\b9353bdd11d8b8cb.jld2` … done (10.9 s)


In [60]:
(conns, STAs, shuffled_STAs) = out;

In [59]:
Base.summarysize(out) / GB

3.18

In [118]:
# Print info on output of `get_connection_to_test()`
function summarize_conns_to_test(ctt)
    n_post = length(unique(ctt.post))
    println("We test $(nrow(ctt)) putative input connections to $(n_post) neurons.")
    n(typ) = count(ctt.conntype .== typ)
    println("$(n(:exc)) of those connections are excitatory, $(n(:inh)) are inhibitory, "*
            "and the remaining $(n(:unconn)) are non-connections.")
end;

In [119]:
summarize_conns_to_test(conns)

We test 3906 putative input connections to 50 neurons.
1515 of those connections are excitatory, 391 are inhibitory, and the remaining 2000 are non-connections.


In [67]:
testall(f; α) = test_conns(f, conns, STAs, shuffled_STAs; α);

In [69]:
tc_ptp = testall(test_conn__ptp, α = 0.01);

Testing connections: 100%|██████████████████████████████| Time: 0:00:01


In [ ]:
function summarize_test_results(tc, typpe)
    pm = perfmeasures(tc)
    i = findfirst(pm.conntypes[typpe])
end

In [75]:
pm = perfmeasures(tc_ptp)

(counts = [1843 576 56; 46 789 10; 111 150 325], sensitivities = Any[0.921, 0.521, 0.831], precisions = Any[0.745, 0.934, 0.555], conntypes = [:unconn, :exc, :inh], conntypes_matrix = [(:unconn, :unconn) (:unconn, :exc) (:unconn, :inh); (:exc, :unconn) (:exc, :exc) (:exc, :inh); (:inh, :unconn) (:inh, :exc) (:inh, :inh)])

In [64]:
perftable(tc_ptp)

Tested connections: 3906,,,,,,
,,┌───────,Real type,───────┐,,Precision
,,unconn,exc,inh,,
┌,unconn,1843,576,56,,74%
Predicted type,exc,46,789,10,,93%
└,inh,111,150,325,,55%
,,,,,,
Sensitivity,,92%,52%,83%,,


```
- 845 exc connections found.
    - 93% of those are correct.
    - 52% of all true exc connections were detected.
```

In [74]:
length(unique(tc_ptp.post))

50